## Traffic Data Predictions

In this file, we use the same methodology as we did for the training dataset, to generate the number of intersections feature for the neighborhoods where there were no monitoring stations.

In [2]:
#Import python packages including overpy
import overpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
import geopy.distance
import math
import time
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas
import folium
import us
from matplotlib import cm
import matplotlib.lines as mlines

sns.set(style = 'whitegrid')
sns.set_palette('bright')
%matplotlib inline

In [10]:
# Load traffic intersection data and clean it
traffic_df = pd.read_csv("../../data/raw/intersection_locations.csv")
#Drop the first column
traffic_df.drop(columns = ['Unnamed: 0'], inplace=True)
## Rename index and intersection number
traffic_df.rename(columns = {'index':'Intersection'}, inplace=True)
### Add an empty column for distance
traffic_df['dist'] = 0.0

# Create individual dataframes
traffic_lat = traffic_df.filter(['Intersection','Latitude'], axis=1)
traffic_long = traffic_df.filter(['Intersection','Longitude'], axis=1)
traffic_dist = traffic_df.filter(['Intersection','dist'], axis=1)

# Transpose all the dataframes
traffic_lat = traffic_lat.T
traffic_long = traffic_long.T
traffic_dist  = traffic_dist.T

#Add suffix to column header based on the dataframe type
traffic_lat.columns = [str(col) + '_latitude' for col in traffic_lat.columns]
traffic_long.columns = [str(col) + '_longitude' for col in traffic_long.columns]
traffic_dist.columns = [str(col) + '_distance' for col in traffic_dist.columns]

## Remove index for each dataframe
traffic_lat.reset_index(drop=True, inplace=True)
traffic_long.reset_index(drop=True, inplace=True)
traffic_dist.reset_index(drop=True, inplace=True)

### Combine individual dataframes into one
traffic_combined = traffic_lat.join(traffic_long).join(traffic_dist)

### Sort based on column names
traffic_combined = traffic_combined.reindex(columns=sorted(traffic_combined.columns))
#Update dataframe to contain 222 rows because there are 222 rows in predict.csv
traffic_combined = traffic_combined.loc[traffic_combined.index.repeat(222)].reset_index(drop=True)

# Load Data for Neighborhoods where we will predict 

In [11]:
neighbourhhood_predict = pd.read_csv("../../data/cleaned/predict.csv")

In [12]:
neighbourhhood_predict.shape

(222, 3)

In [13]:
neighbourhhood_predict.head()

,Unnamed: 0,lat,lon
0,0,29.672427,-95.319700
1,1,29.672427,-95.322203
2,2,29.672427,-95.324705
3,3,29.672427,-95.327208
4,4,29.672427,-95.329710


In [14]:
predict_df = neighbourhhood_predict[['lon', 'lat']]

# Combine predict neighbourhoods data with intersections data

In [15]:
combined_predict_traffic = predict_df.join(traffic_combined)
combined_predict_traffic.head()

,lon,lat,0_distance,0_latitude,0_longitude,1000_distance,1000_latitude,1000_longitude,1001_distance,1001_latitude,...,998_longitude,999_distance,999_latitude,999_longitude,99_distance,99_latitude,99_longitude,9_distance,9_latitude,9_longitude
0,-95.319700,29.672427,0.0,29.680002,-95.458484,0.0,29.744669,-95.410643,0.0,29.668656,...,-95.467527,0.0,29.796032,-95.409997,0.0,29.748337,-95.361603,0.0,29.718541,-95.408966
1,-95.322203,29.672427,0.0,29.680002,-95.458484,0.0,29.744669,-95.410643,0.0,29.668656,...,-95.467527,0.0,29.796032,-95.409997,0.0,29.748337,-95.361603,0.0,29.718541,-95.408966
2,-95.324705,29.672427,0.0,29.680002,-95.458484,0.0,29.744669,-95.410643,0.0,29.668656,...,-95.467527,0.0,29.796032,-95.409997,0.0,29.748337,-95.361603,0.0,29.718541,-95.408966
3,-95.327208,29.672427,0.0,29.680002,-95.458484,0.0,29.744669,-95.410643,0.0,29.668656,...,-95.467527,0.0,29.796032,-95.409997,0.0,29.748337,-95.361603,0.0,29.718541,-95.408966
4,-95.329710,29.672427,0.0,29.680002,-95.458484,0.0,29.744669,-95.410643,0.0,29.668656,...,-95.467527,0.0,29.796032,-95.409997,0.0,29.748337,-95.361603,0.0,29.718541,-95.408966


In [16]:
combined_predict_traffic.shape

(222, 11861)

# Calculate distance between monitoring location and each traffic intersection

Calculate the distance between the latitude and longitude of the monitoring location from EDF data, and the latitude and longitdue of the intersection using the distance function defined below. The function uses two GPS coordinates to calculate geodesic distance.

In [17]:
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [19]:
time1 = time.time()
for index, row in combined_predict_traffic.iterrows():
    for idx, col in enumerate(combined_predict_traffic.columns):
        if "_dist" in col:
            combined_predict_traffic.at[index,col] = distance((row.iloc[1], row.iloc[0]), (row.iloc[idx+1], row.iloc[idx+2]))*3280.84
    if index % 1000 == 0:
        print(f"Completed {index} row")
time2 = time.time()         
print(time2 - time1)

Completed 0 row
28.01495909690857


In [20]:
combined_predict_traffic.head()

,lon,lat,0_distance,0_latitude,0_longitude,1000_distance,1000_latitude,1000_longitude,1001_distance,1001_latitude,...,998_longitude,999_distance,999_latitude,999_longitude,99_distance,99_latitude,99_longitude,9_distance,9_latitude,9_longitude
0,-95.319700,29.672427,44076.178879,29.680002,-95.458484,39050.651974,29.744669,-95.410643,26422.735662,29.668656,...,-95.467527,53399.938151,29.796032,-95.409997,30711.301636,29.748337,-95.361603,32912.859967,29.718541,-95.408966
1,-95.322203,29.672427,43284.410310,29.680002,-95.458484,38469.129209,29.744669,-95.410643,25630.442109,29.668656,...,-95.467527,52979.443953,29.796032,-95.409997,30376.855379,29.748337,-95.361603,32233.649943,29.718541,-95.408966
2,-95.324705,29.672427,42493.017317,29.680002,-95.458484,37895.509818,29.744669,-95.410643,24838.535860,29.668656,...,-95.467527,52567.709345,29.796032,-95.409997,30059.740912,29.748337,-95.361603,31560.026951,29.718541,-95.408966
3,-95.327208,29.672427,41701.370666,29.680002,-95.458484,37329.689138,29.744669,-95.410643,24046.390895,29.668656,...,-95.467527,52164.608777,29.796032,-95.409997,29760.254891,29.748337,-95.361603,30891.800495,29.718541,-95.408966
4,-95.329710,29.672427,40910.106551,29.680002,-95.458484,36772.489619,29.744669,-95.410643,23254.648114,29.668656,...,-95.467527,51770.669733,29.796032,-95.409997,29479.177400,29.748337,-95.361603,30229.879991,29.718541,-95.408966


# Clean Traffic+Distance data

In [25]:
#Drop the latitude column
combined_predict_traffic = combined_predict_traffic[combined_predict_traffic.columns.drop(list(combined_predict_traffic.filter(regex='_latitude')))]
#Drop the longitude column
combined_predict_traffic = combined_predict_traffic[combined_predict_traffic.columns.drop(list(combined_predict_traffic.filter(regex='_longitude')))]

In [26]:
combined_predict_traffic.shape

(222, 3955)

In [24]:
combined_predict_traffic.head()

,lon,lat,0_distance,1000_distance,1001_distance,1002_distance,1003_distance,1004_distance,1005_distance,1006_distance,...,992_distance,993_distance,994_distance,995_distance,996_distance,997_distance,998_distance,999_distance,99_distance,9_distance
0,-95.319700,29.672427,44076.178879,39050.651974,26422.735662,37747.189479,35752.635050,67135.229409,10387.057362,10433.750288,...,51165.234532,48541.657190,47898.735639,47353.219774,47357.565628,47264.971796,47245.647749,53399.938151,30711.301636,32912.859967
1,-95.322203,29.672427,43284.410310,38469.129209,25630.442109,37169.340197,35369.399302,66346.779003,11101.035838,11148.428400,...,50436.963692,47774.378148,47121.442736,46566.681760,46570.911465,46478.461667,46459.023097,52979.443953,30376.855379,32233.649943
2,-95.324705,29.672427,42493.017317,37895.509818,24838.535860,36599.781106,35000.084233,65558.760718,11824.843220,11872.810034,...,49710.968944,47008.267479,46345.002008,45780.691934,45784.801546,45692.501125,45672.944256,52567.709345,30059.740912,31560.026951
3,-95.327208,29.672427,41701.370666,37329.689138,24046.390895,36038.439292,34644.835211,64770.548923,12557.308098,12605.751157,...,48986.758275,46242.755647,45568.820505,44994.634275,44998.619545,44906.474271,44886.795039,52164.608777,29760.254891,30891.800495
4,-95.329710,29.672427,40910.106551,36772.489619,23254.648114,35486.163787,34304.377385,63982.777916,13296.463251,13345.304505,...,48265.003668,45478.500000,44793.548015,44209.150005,44213.006562,44121.022407,44101.216640,51770.669733,29479.177400,30229.879991


# Count the number of intersections with distance less than 1000 ft

In [31]:
#Create an empty column for number of intersection
combined_predict_traffic['number_intersections'] = 0

def count_values_in_range(series, range_min, range_max):
    # "between" returns a boolean Series equivalent to left <= series <= right.
    # NA values will be treated as False.
    return series.between(left=range_min, right=range_max).sum()

range_min, range_max = 0, 1000

combined_predict_traffic['number_intersections'] = combined_predict_traffic.iloc[:,2:].apply(
    func=lambda row: count_values_in_range(row, range_min, range_max), axis=1)


#Keep only coordinates of the monitoring station and number of intersections for each in the final dataset
combined_predict_traffic = combined_predict_traffic[['lat','lon','number_intersections']]

In [32]:
combined_predict_traffic.describe()

,lat,lon,number_intersections
count,222.000000,222.000000,222.000000
mean,29.715005,-95.418311,2.436937
std,0.039068,0.082467,2.506658
min,29.672427,-95.526338,1.000000
25%,29.677936,-95.503857,1.000000
50%,29.686200,-95.364746,1.000000
75%,29.755712,-95.342223,3.000000
max,29.763409,-95.319700,13.000000


In [33]:
combined_predict_traffic.head()

,lat,lon,number_intersections
0,29.672427,-95.319700,1
1,29.672427,-95.322203,1
2,29.672427,-95.324705,1
3,29.672427,-95.327208,1
4,29.672427,-95.329710,1


In [34]:
#Write to a csv file
combined_predict_traffic.to_csv("../../data/cleaned/traffic_data_predict.csv")